In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt

In [2]:
weather_path = "../data/HourlyWeatherData.csv"
weatherDF = pd.read_csv(weather_path)
weatherDF


,DATE,HourlyDryBulbTemperature,HourlyPrecipitation,HourlyRelativeHumidity
0,2017-07-01T00:54:00,75.0,0.0,88.0
1,2017-07-01T01:54:00,76.0,0.0,88.0
2,2017-07-01T02:54:00,75.0,0.0,90.0
3,2017-07-01T03:54:00,75.0,0.0,90.0
4,2017-07-01T04:54:00,75.0,0.0,90.0
...,...,...,...,...
36307,2021-08-22T22:54:00,73.0,0.0,87.0
36308,2021-08-22T23:54:00,71.0,0.0,96.0
36309,2021-08-23T00:54:00,72.0,0.0,94.0
36310,2021-08-23T01:54:00,71.0,NaN,96.0


In [3]:
weatherDF['DATE'] = pd.to_datetime(weatherDF['DATE'])
weatherDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36312 entries, 0 to 36311
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   DATE                      36312 non-null  datetime64[ns]
 1   HourlyDryBulbTemperature  36310 non-null  float64       
 2   HourlyPrecipitation       36109 non-null  float64       
 3   HourlyRelativeHumidity    36310 non-null  float64       
dtypes: datetime64[ns](1), float64(3)
memory usage: 1.1 MB


In [4]:
# Subtract minutes to get hour values on weather table
weatherDF['DATE'] = weatherDF['DATE'] - dt.timedelta(minutes = 54)
weatherDF

,DATE,HourlyDryBulbTemperature,HourlyPrecipitation,HourlyRelativeHumidity
0,2017-07-01 00:00:00,75.0,0.0,88.0
1,2017-07-01 01:00:00,76.0,0.0,88.0
2,2017-07-01 02:00:00,75.0,0.0,90.0
3,2017-07-01 03:00:00,75.0,0.0,90.0
4,2017-07-01 04:00:00,75.0,0.0,90.0
...,...,...,...,...
36307,2021-08-22 22:00:00,73.0,0.0,87.0
36308,2021-08-22 23:00:00,71.0,0.0,96.0
36309,2021-08-23 00:00:00,72.0,0.0,94.0
36310,2021-08-23 01:00:00,71.0,NaN,96.0


In [5]:
salespath = "../data/HourlySalesFullNoCovid.csv" 
salesDF = pd.read_csv(salespath)
salesDF["Business Hour"] = salesDF["Business Hour"].str[:8]


In [6]:
salesDF['DateTime'] = pd.to_datetime(salesDF['Business Date'] + " " + salesDF['Business Hour'])
salesDF = salesDF.replace(',','', regex=True)
salesDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58237 entries, 0 to 58236
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Business Date  58237 non-null  object        
 1   Business Hour  58237 non-null  object        
 2   Sales          58237 non-null  object        
 3   Checks         58237 non-null  int64         
 4   DateTime       58237 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 2.2+ MB


In [7]:
salesDF['Sales']= salesDF['Sales'].apply(pd.to_numeric,errors='coerce')

In [8]:
salesDF = salesDF.drop(['Business Date','Business Hour'], axis=1)

In [9]:
groupedSalesDF = salesDF.groupby(['DateTime']).sum()


In [10]:
WeatherSalesDF = groupedSalesDF.merge(weatherDF, how="inner", left_on="DateTime", right_on= "DATE")
WeatherSalesDF = WeatherSalesDF.drop(['DATE'], axis=1)
WeatherSalesDF

,Sales,Checks,HourlyDryBulbTemperature,HourlyPrecipitation,HourlyRelativeHumidity
0,34.98,6,86.0,0.0,65.0
1,69.36,10,86.0,0.0,65.0
2,72.88,6,90.0,0.0,59.0
3,84.89,12,90.0,0.0,58.0
4,103.16,5,89.0,0.0,57.0
...,...,...,...,...,...
15520,87.02,15,92.0,0.0,46.0
15521,92.81,13,92.0,0.0,47.0
15522,116.08,17,91.0,0.0,45.0
15523,253.88,23,90.0,0.0,44.0


In [11]:
WeatherSalesDF.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15525 entries, 0 to 15524
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Sales                     15525 non-null  float64
 1   Checks                    15525 non-null  int64  
 2   HourlyDryBulbTemperature  15525 non-null  float64
 3   HourlyPrecipitation       15435 non-null  float64
 4   HourlyRelativeHumidity    15525 non-null  float64
dtypes: float64(4), int64(1)
memory usage: 727.7 KB
